In [17]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
import os
from datetime import datetime
from IPython.display import Markdown, display

user_secrets = UserSecretsClient()
local_debug = True
os.environ['HF_TOKEN'] = user_secrets.get_secret("HF_TOKEN")

def connect_to_huggingface():
    hf_token_val = os.getenv('HF_TOKEN')
    login(token=hf_token_val)
    if local_debug :
        print(f"hf_token_val len", len(hf_token_val))
connect_to_huggingface()

def printmd(string):
    display(Markdown(string))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


hf_token_val len 37


In [18]:
detailed_prompt = """
You are an expert in LotusScript and HCL Domino.

Explain the purpose and behavior of the following LotusScript agent.
List:
- What the code does
- Important side effects
- External dependencies
- Potential risks or failure points

<<CODE>>
Option Public
Use "IssueUtils"
Use "NotificationUtils"
Sub Initialize
    On Error GoTo ErrHandler
    Dim session As New NotesSession
    Dim db As NotesDatabase
    Set db = session.CurrentDatabase

    Dim doc As NotesDocument
    Set doc = db.CreateDocument()
    doc.Form = "IssueForm"
    doc.IssueID = GenerateIssueID(session)
    doc.Title = "Sample issue created by CreateIssueAgent"
    Dim rtitem As NotesRichTextItem
    Set rtitem = New NotesRichTextItem(doc, "Description")
    Call rtitem.AppendText("This issue was created by CreateIssueAgent for testing.")
    doc.Priority = "Medium"
    doc.Status = "New"
    doc.CreatedBy = session.CommonUserName
    doc.CreatedDate = Now
    doc.AssignedTo = session.CommonUserName
    doc.NotifyOnCreate = "1"
    Call doc.Save(True, False)

    If (doc.NotifyOnCreate(0) = "1") Then
        Call SendIssueNotification(db, doc.AssignedTo(0), "New Issue Assigned: " & doc.IssueID(0), "Title: " & doc.Title(0))
    End If

    Messagebox "Issue created: " & doc.IssueID(0), , "CreateIssueAgent"
    Exit Sub
ErrHandler:
    Messagebox "Error in CreateIssueAgent: " & Err & " - " & Error$, , "Agent Error"
End Sub
<<CODE>>
"""

In [19]:
    from transformers import AutoModelForCausalLM, AutoTokenizer
    import torch

    model_name = "codellama/CodeLlama-7b-Instruct-hf"

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_8bit=True,
        device_map="auto",
        torch_dtype=torch.float16  # Use float16 for reduced memory footprint
    )

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
start_time = datetime.now()
if local_debug :
    print("start_time: ",start_time)
prompt = detailed_prompt
data_str = "llm_output"
#pt - pytorch
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

output = model.generate(
    inputs["input_ids"],
    max_new_tokens=180,
    do_sample=True,
    temperature=0.7, # Creativity
    top_k=50, # Select top k tokens
    top_p=0.95 # Select top tokens whos probability is more than threshold p
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

end_time = datetime.now()
elapsed_time = end_time - start_time
if local_debug :
    print("end_time: ",end_time)     
    print(f"Elapsed time for {data_str}: {elapsed_time.total_seconds():.4f} seconds which is equal to  {elapsed_time.total_seconds()/60:.4f} minutes")
print(generated_text)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


start_time:  2025-12-04 03:41:25.239916
end_time:  2025-12-04 03:41:56.084770
Elapsed time for llm_output: 30.8449 seconds which is equal to  0.5141 minutes

You are an expert in LotusScript and HCL Domino.

Explain the purpose and behavior of the following LotusScript agent.
List:
- What the code does
- Important side effects
- External dependencies
- Potential risks or failure points

<<CODE>>
Option Public
Use "IssueUtils"
Use "NotificationUtils"
Sub Initialize
    On Error GoTo ErrHandler
    Dim session As New NotesSession
    Dim db As NotesDatabase
    Set db = session.CurrentDatabase

    Dim doc As NotesDocument
    Set doc = db.CreateDocument()
    doc.Form = "IssueForm"
    doc.IssueID = GenerateIssueID(session)
    doc.Title = "Sample issue created by CreateIssueAgent"
    Dim rtitem As NotesRichTextItem
    Set rtitem = New NotesRichTextItem(doc, "Description")
    Call rtitem.AppendText("This issue was created by CreateIssueAgent for testing.")
    doc.Priority = "Medium"